nacitanie dat, spojenie new_obesrvation s new_station
rozdelenie na trainig data a testing data casti
vsetky atribute normlaizovat aby boli numericke -> vhodne pre ML (pricom kategoricke features  budu continenty, vynechavame staty a mesta)

